## Etude des sujets des amendements PLFSS - Sasha

## Préparation des données

In [1]:
#!python -m pip install --upgrade pip

In [2]:
#!pip install -q --upgrade pandas  spacy  more_itertools

In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
#%reload_ext autoreload
#%autoreload 2
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [4]:
#!python -m spacy download fr_core_news_sm > /dev/null

In [5]:
# Importing modules
import pandas as pd
#amdt = pd.read_csv('https://github.com/leximpact/donnees-extraites-assemblee/blob/main/textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdts = pd.read_csv('https://raw.githubusercontent.com/leximpact/donnees-extraites-assemblee/main/textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdts.head(5)

,texteLegislatifUid,uid,avantAApres,dispositif,exposeSommaire
0,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000001,A,"I. – Après l’alinéa 13, insérer l’alinéa suiv...",La mise en place d’un accord d’intéressement d...
1,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000002,Apres,À la première phrase du premier alinéa de l’ar...,L’article L 531‑2 du Code de la Sécurité Socia...
2,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000005,A,Compléter l’alinéa 17 par la phrase suivante :...,S’il est louable d’expérimenter des dispositif...
3,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000008,A,"Après l’alinéa 8, insérer les cinq alinéas sui...",Cet amendement permet d’amplifier la portée de...
4,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000010,Apres,Le premier alinéa de l’article L. 521‑1 du cod...,"Pendant plus de cinquante ans, notre politique..."


In [6]:
print(len(amdts)) #Nombre total d'amendements

4797


In [7]:
#On regroupe dans un même texte chaque dispositif et son exposé sommaire
amdts['texte'] = amdts['dispositif'] + amdts['exposeSommaire'] 

# Nettoyage via le pipeline spacy : 
##  Step 1 : transformer, morphologizer, parser, attribute_ruler, lemmatizer

In [8]:
import spacy

#Choix du package
nlp = spacy.load("fr_core_news_sm") #python -m spacy download fr_core_news_sm #Moins précis 
#nlp = spacy.load("fr_core_news_md") #python -m spacy download fr_core_news_md
#nlp = spacy.load("fr_core_news_lg") #python -m spacy download fr_core_news_lg
#nlp = spacy.load("fr_dep_news_trf") #python -m spacy download fr_dep_news_trf

# LIGNE A COMMENTER SI ON VEUT FAIRE LE CALCUL SUR L'ENSEMBLE DU CORPUS
#amdts = amdts[0:6]

lemmatized_amdts = []
amdt = []
for amdt in amdts['texte']:
    lemmatized_amdt=[]
    token = []
    for token in nlp(amdt):
        lemmatized_amdt.append(token.lemma_)
        #print(token.text,token.lemma_)
    lemmatized_amdts.append(lemmatized_amdt)

#print('AVANT:', amdts['texte'][4])    
#print('APRES:', lemmatized_amdts[4])
print(len(lemmatized_amdts))

4797


##  Step 2 : remove casing and punctuation

In [9]:
## Removing punctuation AND Casing
#( Casing: Est-ce vraiment utile ? Est-ce qu'on ne va pas perdre les Acronymes de vue ?)
amdts_clean = []
for amdt in lemmatized_amdts:
    amdts_clean.append([ word.lower() for word in amdt if word.isalpha()])
print(amdts_clean[4])
print(len(amdts_clean))

['le', 'premier', 'alinéa', 'de', 'article', 'de', 'code', 'de', 'le', 'sécurité', 'social', 'être', 'compléter', 'par', 'un', 'phrase', 'ainsi', 'rédiger', 'lui', 'être', 'universel', 'pendant', 'plus', 'de', 'cinquante', 'an', 'notre', 'politique', 'familial', 'avoir', 'reposer', 'sur', 'le', 'principe', 'de', 'universalité', 'cela', 'signifier', 'lui', 'adresser', 'à', 'tout', 'le', 'français', 'sans', 'distinction', 'social', 'lui', 'reposer', 'sur', 'idée', 'que', 'chaque', 'enfant', 'à', 'naître', 'être', 'un', 'chance', 'et', 'un', 'richesse', 'pour', 'le', 'france', 'pour', 'son', 'avenir', 'quel', 'que', 'être', 'le', 'ressource', 'dont', 'disposer', 'le', 'parent', 'pour', 'mettre', 'en', 'place', 'ce', 'principe', 'universalité', 'le', 'politique', 'familial', 'appeler', 'de', 'outil', 'dire', 'de', 'redistribution', 'horizontal', 'être', 'à', 'dire', 'un', 'mécanisme', 'de', 'solidarité', 'de', 'famille', 'sans', 'enfant', 'envers', 'le', 'famille', 'avec', 'enfant', 'pour'

##  Step 3 : remove stopwords

In [10]:
#pip install nltk

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/sasha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sasha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
#STOP WORDS
import spacy
#On importe les mots "inutiles" (stopwords) du langage français depuis Space -ET- NLTK
from nltk.corpus import stopwords 
stop_words = stopwords.words("french")
#print(stop_words)

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
#print(fr_stop)
for word in fr_stop:
        stop_words.append(word)
del(word)
stop_words = list(stop_words)

#On importe nos propres stopwords depuis le fichier CSV
SW = pd.read_csv('Added_stop_words.csv')
SW = list(SW.StopWords)
final_SW = stop_words+ SW

#On met tout en minuscule et on enlève les éventuels espaces (avant et après chaque mot)
final_SW = [ word.lower().strip('') for word in final_SW]
      
#On enlève les doublons
df = pd.DataFrame({"StopWords" : final_SW})
final_SW = df.drop_duplicates()
final_SW = list(final_SW.StopWords)
print("NOS MOTS 'INUTILES' : \n \n", final_SW ,'\n')

#STOP WORD REMOVAL
print(len(amdts_clean))
print("AVANT : \n \n", amdts_clean[4], "\n")
tokenized = []
for amdt in amdts_clean:
    tokenized.append([ word for word in amdt if word not in final_SW])
print("APRÈS : \n \n", tokenized[4], "\n")
print(len(tokenized))

NOS MOTS 'INUTILES' : 
 
 ['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'aura

##  Step 4 : remove duplicates and save the cleaned data

In [13]:
#On enregistre le corpus d'amendements lemmatizé et sans stopwords
df = pd.DataFrame(data = tokenized )
L1 = len(df)
df.drop_duplicates()
print("Nombre d'amendements en double retirés : ", L1-len(df), "\n")
df = df.fillna('') 
print(df.head(10))
df.to_csv('data_csv/amdts_cleaned.csv')  #Chaque ligne est un amendement
amdts_cleaned = tokenized #pour la suite du code

Nombre d'amendements en double retirés :  0 

        0             1             2            3           4     \
0    insérer    entreprise       salarié    employeur   autoriser   
1       code      sécurité        social    attribuer     insérer   
2  compléter      remettre     parlement     échéance     période   
3    insérer          code       limiter  possibilité   organiser   
4       code      sécurité        social    compléter     rédiger   
5       mois       compter  promulgation     remettre   parlement   
6       mois  promulgation      remettre    parlement      évalue   
7       mois  promulgation      remettre    parlement      évalue   
8  améliorer         prise        charge         pied  diabétique   
9  supprimer   financement      sécurité       social     prévoir   

              5            6            7                8         9     ...  \
0        attribuer           an     ensemble          salarié  employer  ...   
1           verser         code   

# Les mots les plus utilisés

In [14]:
#Counting
from collections import Counter
print(len(amdts_cleaned))
def most_common_words(amdts_cleaned, N):
    freq = []
    words = []
    token = []
    for token in amdts_cleaned:
        bow = Counter(token) 
        #print(bow.most_common(N))
        words.append([bow.most_common(N)[i][0] for i in range(len(bow.most_common(N)))])
        freq.append(bow.most_common(N))  #([bow.most_common(N)[i][1] for i in range(len(bow.most_common(N)))]) 
    return pd.DataFrame(freq), words

#On ne garde que les N mots les plus utilisés pour chaque amendement
N = 10
freq, words = most_common_words(amdts_cleaned, N)

#On sauvegarde cette donnée dans un fichier .csv dont les lignes sont les amdts et les colonnes sont les mots et leur fréquence
#Nom des colonnes (mots)
freq_index = ['Mot Nº{i}'.format(i = j+1) for j in range(N)]
freq.columns = freq_index

#Nom des lignes (amendements)
corpus_index = amdts['uid'].tolist()
freq.index = corpus_index

freq.to_csv('data_csv/most_common_{nb}_words_per_amdt.csv'.format(nb = N))

print("Ci-dessous un exemple des {N} mots les plus utilisés sur 8 amendements: \n \n ".format(N = N), freq.head(8))

4797
Ci-dessous un exemple des 10 mots les plus utilisés sur 8 amendements: 
 
                                                  Mot Nº1             Mot Nº2  \
AMANR5L15PO420120B2296P0D1N000001          (salarié, 6)     (entreprise, 5)   
AMANR5L15PO420120B2296P0D1N000002       (naissance, 15)        (enfant, 15)   
AMANR5L15PO420120B2296P0D1N000005  (expérimentation, 5)           (code, 3)   
AMANR5L15PO420120B2296P0D1N000008          (social, 10)  (établissement, 7)   
AMANR5L15PO420120B2296P0D1N000010         (familial, 7)      (politique, 5)   
AMANR5L15PO420120B2296P0D1N000011         (familial, 8)         (revenu, 8)   
AMANR5L15PO420120B2296P0D1N000012         (familial, 4)     (allocation, 4)   
AMANR5L15PO420120B2296P0D1N000015         (familial, 4)     (allocation, 4)   

                                           Mot Nº3               Mot Nº4  \
AMANR5L15PO420120B2296P0D1N000001   (employeur, 3)            (prime, 3)   
AMANR5L15PO420120B2296P0D1N000002       (prime, 9)     

In [21]:
# Analyse des mots du corpus dans sa globalité
words = pd.DataFrame(words)
mots_uniques = []
for c in words:
    temp = list (words[c].unique())
    for word in temp:
        if type(word) is not float:
            mots_uniques.append( str(word) )  #List of unique words
        
#On enlève les doublons
mots_uniques = list(dict.fromkeys(mots_uniques))
mots_uniques.sort()
print( "Nombre total de mots les plus utilisés : ", len(mots_uniques), '\n' )
print("Liste des mots les plus utilisés: ", mots_uniques)

Nombre total de mots les plus utilisés :  2466 

Liste des mots les plus utilisés:  ['None', 'aa', 'aah', 'ab', 'abaisse', 'abandon', 'abattement', 'abattemer', 'aboutir', 'abroger', 'abrogé', 'abrogés', 'absence', 'académique', 'acceptable', 'accepter', 'accessibler', 'accessoire', 'accident', 'accidenté', 'accise', 'accompagnement', 'accompagnemer', 'accompagner', 'accomplissement', 'accord', 'accorder', 'accouchement', 'accroître', 'accru', 'accréditation', 'accueil', 'accueillir', 'accès', 'accéder', 'achat', 'acos', 'acquisition', 'acquitter', 'acre', 'act', 'acte', 'acteur', 'actif', 'action', 'actionnaire', 'activité', 'actualisation', 'actualiser', 'actuel', 'actuellement', 'adaptation', 'adapter', 'additif', 'additionnel', 'administratif', 'administration', 'adolescent', 'adopter', 'adoption', 'adresser', 'adulte', 'aeeh', 'affaiblir', 'affaire', 'affectation', 'affecter', 'affection', 'affiliation', 'affilie', 'affilier', 'affilié', 'affirmer', 'afférent', 'afp', 'agence', 'a

# TF IDF
Term frequency - inverse document frequency

## Calculs et analyse

In [51]:
import sklearn
import pickle
#Mise sous forme 'corpus' (une liste de tous les textes)
corpus = []
for amdt1 in amdts_cleaned:
    temp = ' '.join(amdt1)
    corpus.append(temp)
    temp = ''
    
#Vectorization - Term Frequency in Global Corpus
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
tf = vectorizer.fit_transform(corpus)

#Sauvegarde dans un fichier csv
with open("data_csv/corpus.txt", "wb") as fp:   #Pickling
    pickle.dump(corpus, fp)

print(corpus[2:4])

['compléter remettre parlement échéance période expérimentation évaluation porter contemporanéité crédit impôt mentionner sexdecie code général impôt participation financier bénéficiaire prestation mentionner code action social famille part coût induire application degré prestataire définir degré code travail participer expérimentation part louable expérimenter dispositif permettre décalage moment dépense réaliser aide percevoir recourir service domicile vulnérable nécessaire mesurer éventuel induire négatif saad facturer prix participation financier légal prévoir apa pch convier mesurer expérimentation prix service conséquence charge ailleurs expérimentation prévoit contemporanéité apa pch organiser centre cesu déjà conseil départemental pouvoir verser mécanisme tiers payer prise charge saad limiter avance trésorerie bénéficiaire convier vérifier expérimentation introduire nouveau acteur engendre surcoût service prestataire complexité supplémentaire', 'insérer code limiter possibilité

In [23]:
#Features: l'ensemble des mots dans le corpus (après nettoyage)
feature_names = vectorizer.get_feature_names()
print(feature_names[:10])

['aa', 'aah', 'aasctel', 'ab', 'abaisse', 'abaissement', 'abaisser', 'abandon', 'abandonne', 'abandonner']


In [27]:
#IDF - Inverse Document Frequencies
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

#Initialize and fit TfidfVectorizer
vectorizer = TfidfVectorizer(norm=None)
tf_idf_scores = vectorizer.fit_transform(corpus)

#Résultats sous forme matricielle
df_tf_idf = pd.DataFrame(tf_idf_scores.todense())
df_tf_idf.columns =  feature_names
df_tf_idf.index = corpus_index
print("Matrice de densité: poids de chaque mot dans chacun des amendements", df_tf_idf.head(), "\n \n")

#Sauvegarde dans un fichier csv
df_tf_idf.to_csv('data_csv/matrice_de_densite_par_rapport_au_corpus.csv'.format(nb = N))

Matrice de densité: poids de chaque mot dans chacun des amendements                                     aa  aah  aasctel   ab  abaisse  \
AMANR5L15PO420120B2296P0D1N000001  0.0  0.0      0.0  0.0      0.0   
AMANR5L15PO420120B2296P0D1N000002  0.0  0.0      0.0  0.0      0.0   
AMANR5L15PO420120B2296P0D1N000005  0.0  0.0      0.0  0.0      0.0   
AMANR5L15PO420120B2296P0D1N000008  0.0  0.0      0.0  0.0      0.0   
AMANR5L15PO420120B2296P0D1N000010  0.0  0.0      0.0  0.0      0.0   

                                   abaissement  abaisser  abandon  abandonne  \
AMANR5L15PO420120B2296P0D1N000001          0.0       0.0      0.0        0.0   
AMANR5L15PO420120B2296P0D1N000002          0.0       0.0      0.0        0.0   
AMANR5L15PO420120B2296P0D1N000005          0.0       0.0      0.0        0.0   
AMANR5L15PO420120B2296P0D1N000008          0.0       0.0      0.0        0.0   
AMANR5L15PO420120B2296P0D1N000010          0.0       0.0      0.0        0.0   

                              

## Enregistrement des mots les plus importants pour chaque amendement par rapport au reste du corpus

In [28]:
#On analyse pour chaque amendement les X mots les plus importants (par rapport au reste du corpus)
outstanding_words = []
for i in range(len(df_tf_idf)): #On parcourt les amendements
    mots = []
    poids = []
    mots_sorted = []
    for mot in df_tf_idf :
        if df_tf_idf[mot].iloc[i] > 0 :
            mots.append(str(mot)) # (mot, poids)
            poids.append(df_tf_idf[mot].iloc[i]) # (mot, poids)
    mots_sorted = [x for _,x in sorted(zip(poids,mots))]
    print(mots_sorted)
    outstanding_words.append(mots_sorted)       
     
del(df)
df = pd.DataFrame(outstanding_words)
df.index = corpus_index
print(df.head())  

#Sauvegarde dans un fichier csv
df.to_csv('data_csv/mots_importants_specifiques_a_chaque_amdt.csv')

['sécurité', 'vise', 'insérer', 'prévoir', 'compléter', 'état', 'organisme', 'conséquence', 'euro', 'an', 'ensemble', 'cotisation', 'montant', 'contribution', 'social', 'création', 'financier', 'majoration', 'taxe', 'créer', 'code', 'bénéficiaire', 'additionnel', 'pouvoir', 'possibilité', 'revenu', 'limiter', 'limite', 'autoriser', 'versement', 'exonérer', 'salarier', 'ordre', 'technique', 'accorder', 'mentionner', 'attribuer', 'droit', 'général', 'conditionner', 'perte', 'condition', 'recette', 'appréciation', 'mise', 'chef', 'compenser', 'devoir', 'obstacle', 'concurrence', 'intéressement', 'écarter', 'heurte', 'achat', 'place', 'impôt', 'employer', 'intéressemer', 'employeur', 'exceptionnel', 'accord', 'prime', 'entreprise', 'salarié']
['pouvoir', 'santé', 'vise', 'insérer', 'charge', 'disposition', 'dernier', 'cadre', 'ensemble', 'compenser', 'nécessaire', 'besoin', 'viser', 'important', 'ailleurs', 'création', 'numéro', 'modifier', 'mission', 'niveau', 'jour', 'obligation', 'infor

['vise', 'faire', 'prévoir', 'charge', 'compléter', 'national', 'euro', 'prendre', 'objectif', 'fixer', 'application', 'viser', 'contre', 'numéro', 'moyen', 'principe', 'acteur', 'répondre', 'solidarité', 'autonomie', 'base', 'caisse', 'bénéficiaire', 'contexte', 'critère', 'actuel', 'engager', 'venir', 'contrat', 'recours', 'modèle', 'exister', 'concertation', 'enjeu', 'crédit', 'réglementaire', 'inégalité', 'allouer', 'texte', 'rester', 'souhaiter', 'lfs', 'conclure', 'pluriannuel', 'ampleur', 'année', 'fédération', 'différence', 'mai', 'réform', 'démographique', 'opérer', 'poser', 'nouveau', 'octroyer', 'impose', 'majoritairement', 'contentieux', 'modalité', 'cpom', 'sélection', 'institue', 'susviser', 'conserve', 'préfiguration', 'financement', 'préfigure', 'préciser', 'domicile', 'décret', 'tarification', 'million', 'habiliter', 'enveloppe', 'attribution', 'saad']
['vise', 'public', 'degré', 'perte', 'substituer', 'ensemble', 'service', 'viser', 'autonomie', 'vie', 'emploi', 'empl

['santé', 'devoir', 'permettre', 'faire', 'public', 'année', 'nouveau', 'prise', 'travail', 'crise', 'financier', 'décembre', 'créer', 'urgence', 'priver', 'bien', 'sein', 'temps', 'actuel', 'gestion', 'décision', 'revenir', 'fort', 'personnel', 'tard', 'dotation', 'global', 'falloir', 'trop', 'réel', 'attendre', 'sujet', 'présente', 'état', 'efficacité', 'montrer', 'fond', 'activité', 'piste', 'critique', 'lumière', 'abandonner', 'parlement', 'néfaste', 'bénéfique', 'hospitalier', 'financement', 'établissant', 'établissement', 'avèrer', 'remettra', 'tarification', 'avantage', 'mode', 'système', 'inconvénient', 'envisager']
['social', 'pouvoir', 'permettre', 'prévoir', 'dispositif', 'compléter', 'général', 'nouveau', 'conséquence', 'prise', 'travail', 'nécessaire', 'application', 'prestation', 'dépense', 'déjà', 'définir', 'réaliser', 'prévoit', 'remettre', 'ailleurs', 'conseil', 'aide', 'parlement', 'famille', 'supplémentaire', 'porter', 'acteur', 'coût', 'verser', 'évaluation', 'acti

['social', 'pouvoir', 'permettre', 'prévoir', 'dispositif', 'compléter', 'général', 'nouveau', 'conséquence', 'prise', 'travail', 'nécessaire', 'application', 'prestation', 'dépense', 'déjà', 'définir', 'réaliser', 'prévoit', 'remettre', 'ailleurs', 'conseil', 'aide', 'parlement', 'famille', 'supplémentaire', 'porter', 'acteur', 'coût', 'verser', 'évaluation', 'action', 'période', 'mécanisme', 'limiter', 'domicile', 'payer', 'percevoir', 'crédit', 'participer', 'éventuel', 'tiers', 'moment', 'introduire', 'centre', 'organiser', 'mentionner', 'légal', 'charge', 'degré', 'départemental', 'surcoût', 'recourir', 'avance', 'trésorerie', 'expérimenter', 'négatif', 'vérifier', 'code', 'vulnérable', 'échéance', 'facturer', 'impôt', 'complexité', 'engendre', 'décalage', 'louable', 'part', 'financier', 'cesu', 'sexdecie', 'convier', 'bénéficiaire', 'prix', 'participation', 'service', 'mesurer', 'induire', 'pch', 'prestataire', 'saad', 'apa', 'contemporanéité', 'expérimentation']
['devoir', 'insé

['social', 'sécurité', 'code', 'pouvoir', 'vise', 'insérer', 'compléter', 'maladie', 'état', 'assurance', 'entreprise', 'réaliser', 'système', 'très', 'taxe', 'convier', 'emploi', 'second', 'appliquer', 'critère', 'marché', 'point', 'éviter', 'salaire', 'élevé', 'travers', 'production', 'supérieur', 'contribuer', 'occurrence', 'compétitivité', 'largement', 'initial', 'industrie', 'fondamental', 'rembourser', 'investir', 'cice', 'contribue', 'partage', 'risquer', 'phase', 'arriver', 'octroi', 'prendre', 'final', 'contribuable', 'former', 'chercheur', 'subvention', 'détermination', 'aspect', 'indirectement', 'paie', 'enseignemer', 'part', 'cir', 'excellence', 'découverte', 'cit', 'financement', 'impôt', 'définition', 'scientifique', 'public', 'biomédical', 'investissement', 'médicament', 'crédit', 'prix', 'développement', 'recherche']
['santé', 'devoir', 'droit', 'général', 'organisme', 'perte', 'nouveau', 'proposer', 'recette', 'activité', 'prendre', 'entreprise', 'impôt', 'compenser', 

...

['pouvoir', 'permettre', 'faire', 'supprimer', 'insérer', 'proposer', 'conséquence', 'prendre', 'cas', 'objectif', 'cadre', 'assurer', 'médical', 'place', 'bénéficier', 'important', 'ailleurs', 'projet', 'groupe', 'priver', 'vie', 'sein', 'relever', 'période', 'possibilité', 'actuel', 'pratique', 'décision', 'voir', 'expérimentation', 'atteindre', 'suivi', 'parent', 'sénat', 'autoriser', 'soutien', 'falloir', 'lien', 'accompagnement', 'nature', 'association', 'maternité', 'existe', 'associer', 'adoption', 'uniquement', 'respecter', 'débat', 'fondamental', 'appeler', 'repli', 'voire', 'choisir', 'filière', 'aboutir', 'retirer', 'effectivement', 'accouchement', 'gérer', 'esprit', 'nom', 'utile', 'personnaliser', 'privilégier', 'conjoint', 'impose', 'confiance', 'inutile', 'embaucher', 'veiller', 'party', 'version', 'expertise', 'recenser', 'conforter', 'prenant', 'risque', 'quantitatif', 'créer', 'jour', 'salle', 'ambiguïté', 'parentalité', 'physiologique', 'revenir', 'adopter', 'accouch

['pouvoir', 'devoir', 'vise', 'charge', 'financement', 'maladie', 'perte', 'nouveau', 'assurance', 'proposer', 'euro', 'recette', 'part', 'lier', 'principe', 'sécurité', 'milliard', 'convier', 'réforme', 'revenir', 'venir', 'donner', 'atteindre', 'autant', 'correspondre', 'apparaître', 'compensation', 'fixé', 'conformément', 'dette', 'déficit', 'transfert', 'prévoir', 'difficile', 'atteinte', 'état', 'dépendance', 'veil', 'effectivement', 'objectif', 'contrairement', 'dépense', 'social', 'apurement', 'creux', 'déterminant', 'compense', 'politique']
['degré', 'état', 'nouveau', 'titre', 'plfs', 'proposer', 'rejeter']
['santé', 'vise', 'disposition', 'complémentaire', 'actuel', 'rédaction', 'accompagner', 'renouvellement', 'périmètre', 'informer', 'ouverture', 'solidaire', 'ensemble', 'vis', 'inclure', 'bénéficiaire', 'aah']
['état', 'décret', 'fin', 'nécessaire', 'bénéficier', 'conseil', 'durée', 'remplacer', 'fonction', 'temps', 'pouvoir', 'domicile', 'engagement', 'supérieur', 'matern

['pouvoir', 'vise', 'faire', 'supprimer', 'charge', 'perte', 'recette', 'dernier', 'montant', 'prestation', 'économique', 'prévoit', 'financier', 'français', 'coût', 'budget', 'décision', 'adopter', 'avis', 'correspondre', 'achat', 'responsabilité', 'peser', 'complet', 'état', 'existant', 'général', 'sécurité', 'induire', 'veil', 'communication', 'rapporteur', 'estime', 'futur', 'consister', 'distribuer', 'social', 'assume', 'politique', 'rompre', 'urgence', 'approprié', 'correspondent', 'sénat', 'logique', 'décider', 'bdget', 'décideur', 'vanlerenberghe']
['faire', 'supprimer', 'assurance', 'disposition', 'cotisation', 'entreprise', 'relatif', 'contre', 'fonction', 'employeur', 'adopter', 'patronal', 'contrat', 'sénat', 'utilisation', 'respecter', 'sanction', 'favorable', 'chômage', 'opposer', 'revanche', 'malus', 'cdd', 'abusent', 'courts', 'bonus']
['social', 'vise', 'supprimer', 'public', 'cotisation', 'contribution', 'exonération', 'spécifique', 'fonction', 'plfs', 'commission', '

['faire', 'public', 'état', 'cotisation', 'agir', 'contribution', 'exonération', 'social', 'créer', 'groupe', 'spécifique', 'fonction', 'bon', 'socialiste', 'sens', 'budget', 'sénat', 'rupture', 'décider', 'conventionnel', 'supprimer', 'déficit', 'indemnité', 'voter', 'respecter', 'entendre', 'haute', 'ajustement', 'variable', 'espèce', 'vote', 'chambre', 'inviter', 'artificiellement']
['social', 'faire', 'supprimer', 'degré', 'état', 'créer', 'groupe', 'socialiste', 'budget', 'décider', 'déficit', 'entendre', 'ajustement', 'variable', 'artificiellement']
['faire', 'supprimer', 'proposer', 'an', 'prévoit', 'social', 'raison', 'aide', 'groupe', 'milliard', 'appliquer', 'socialiste', 'venir', 'connaître', 'population', 'économie', 'augmenter', 'hauteur', 'représenter', 'fiscal', 'pension', 'budgétaire', 'passer', 'apparenter', 'publier', 'déposer', 'retraité', 'pourcent', 'aggraver', 'stratégie', 'pleinement', 'contribue', 'année', 'mobiliser', 'démontrer', 'enjeux', 'oppose', 'haut', 'l

['supprimer', 'financement', 'droit', 'assurer', 'cotisation', 'agir', 'social', 'sécurité', 'très', 'groupe', 'fonction', 'employeur', 'socialiste', 'rémunération', 'vigueur', 'patronal', 'contrat', 'utilisation', 'unique', 'produire', 'périmètre', 'constitution', 'dispositif', 'contraire', 'plein', 'voire', 'disposition', 'mineur', 'constitutionnel', 'apparenté', 'relatif', 'strictement', 'employé', 'écarter', 'organique', 'artificiel', 'assurance', 'courts', 'malus', 'bonus', 'chômage']
['devoir', 'permettre', 'rédiger', 'prévoir', 'financement', 'droit', 'national', 'organisme', 'perte', 'recette', 'dernier', 'cadre', 'ensemble', 'cotisation', 'impôt', 'compenser', 'concurrence', 'bénéficier', 'création', 'modifier', 'régime', 'porter', 'taxe', 'renforcer', 'code', 'applicable', 'second', 'additionnel', 'contexte', 'réforme', 'face', 'répartition', 'rémunération', 'point', 'total', 'élevé', 'assemblée', 'correspondre', 'sénat', 'production', 'exonérer', 'étendre', 'salarial', 'résu

## Comparatif: mots spécifiques VS mots relatifs

In [49]:
N = 4
df2 = []
for i in range(len(df)):
    mots_spec = []
    mots_rel = []
    for j in range(N):
        if freq.iloc[i][j] != None :
            mots_spec.append( freq.iloc[i][j][0] )
            mots_rel.append( df.iloc[i][j] )
    df2.append( mots_spec + mots_rel ) # N mots relatifs

#Mise en forme     
df2 = pd.DataFrame(df2)
spec_index = ['Mot Fréquent{i}'.format(i = j+1) for j in range(N)]
rel_index = ['Mot Important{i}'.format(i = j+1) for j in range(N)]
df2.columns = [spec_index + rel_index]
df2.index = corpus_index  
print("{nb} mots spécifiques et {nb} mots relatifs : ".format(nb = N), df2.head(), "\n")
#Sauvegarde dans un fichier csv
df2.to_csv('data_csv/comparatif_{nb}_mots_specifiques_et_relatifs_pour_chaque_amdts.csv'.format(nb = N))

4 mots spécifiques et 4 mots relatifs :                                       Mot Fréquent1  Mot Fréquent2  \
AMANR5L15PO420120B2296P0D1N000001          salarié     entreprise   
AMANR5L15PO420120B2296P0D1N000002        naissance         enfant   
AMANR5L15PO420120B2296P0D1N000005  expérimentation           code   
AMANR5L15PO420120B2296P0D1N000008           social  établissement   
AMANR5L15PO420120B2296P0D1N000010         familial      politique   

                                  Mot Fréquent3    Mot Fréquent4  \
AMANR5L15PO420120B2296P0D1N000001     employeur            prime   
AMANR5L15PO420120B2296P0D1N000002         prime           social   
AMANR5L15PO420120B2296P0D1N000005       service  contemporanéité   
AMANR5L15PO420120B2296P0D1N000008       service             code   
AMANR5L15PO420120B2296P0D1N000010        social         principe   

                                  Mot Important1 Mot Important2  \
AMANR5L15PO420120B2296P0D1N000001       sécurité           vise   
A

In [ ]:
#Toutes les données sont exportées en CSV dans le folder data_csv